In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import LabelEncoder
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv('data/sales_train.csv', parse_dates=['date'])
test = pd.read_csv('data/sales_test.csv', parse_dates=['date'])
ss = pd.read_csv('data/solution.csv')
inventory = pd.read_csv('data/inventory.csv')
weights = pd.read_csv('data/test_weights.csv')
calendar = pd.read_csv('data/calendar.csv', parse_dates=['date'])

In [4]:
from datetime import datetime
# 定義節假日
czech_holiday = [ 
    (['03/31/2024', '04/09/2023', '04/17/2022', '04/04/2021', '04/12/2020'], 'Easter Day'),#loss
    (['05/12/2024', '05/10/2020', '05/09/2021', '05/08/2022', '05/14/2023'], "Mother Day"), #loss
]
brno_holiday = [
    (['03/31/2024', '04/09/2023', '04/17/2022', '04/04/2021', '04/12/2020'], 'Easter Day'),#loss
    (['05/12/2024', '05/10/2020', '05/09/2021', '05/08/2022', '05/14/2023'], "Mother Day"), #loss
]

budapest_holidays = []
munich_holidays = [
    (['03/30/2024', '04/08/2023', '04/16/2022', '04/03/2021'], 'Holy Saturday'),#loss
    (['05/12/2024', '05/14/2023', '05/08/2022', '05/09/2021'], 'Mother Day'),#loss
]

frank_holidays = [
    (['03/30/2024', '04/08/2023', '04/16/2022', '04/03/2021'], 'Holy Saturday'),#loss
    (['05/12/2024', '05/14/2023', '05/08/2022', '05/09/2021'], 'Mother Day'),#loss
]
#填充節假日訊息
def fill_loss_holidays(df_fill, warehouses, holidays):
    df = df_fill.copy()
    for item in holidays:
        dates, holiday_name = item
        generated_dates = [datetime.strptime(date, '%m/%d/%Y').strftime('%Y-%m-%d') for date in dates]
        for generated_date in generated_dates:
            df.loc[(df['warehouse'].isin(warehouses)) & (df['date'] == generated_date), 'holiday'] = 1
            df.loc[(df['warehouse'].isin(warehouses)) & (df['date'] == generated_date), 'holiday_name'] = holiday_name
    return df

calendar = fill_loss_holidays(df_fill=calendar, warehouses=['Prague_1', 'Prague_2', 'Prague_3'], holidays=czech_holiday)
calendar = fill_loss_holidays(df_fill=calendar, warehouses=['Brno_1'], holidays=brno_holiday)
calendar = fill_loss_holidays(df_fill=calendar, warehouses=['Munich_1'], holidays=munich_holidays)
calendar = fill_loss_holidays(df_fill=calendar, warehouses=['Frankfurt_1'], holidays=frank_holidays)
calendar = fill_loss_holidays(df_fill=calendar, warehouses=['Budapest_1'], holidays=budapest_holidays)

In [5]:
# 查詢特定倉庫的節假日信息
Frankfurt_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Frankfurt_1"')
Prague_2 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Prague_2"')
Brno_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Brno_1"')
Munich_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Munich_1"')
Prague_3 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Prague_3"')
Prague_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Prague_1"')
Budapest_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Budapest_1"')

# 處理節假日信息
def process_calendar(df):
    df = df.sort_values('date').reset_index(drop=True)
    df['next_holiday_date'] = df.loc[df['holiday'] == 1, 'date'].shift(-1)
    df['next_holiday_date'] = df['next_holiday_date'].bfill()
    df['days_to_holiday'] = (df['next_holiday_date'] - df['date']).dt.days
    df.drop(columns=['next_holiday_date'], inplace=True)
    df['next_shops_closed_date'] = df.loc[df['shops_closed'] == 1, 'date'].shift(-1)
    df['next_shops_closed_date'] = df['next_shops_closed_date'].bfill()
    df['days_to_shops_closed'] = (df['next_shops_closed_date'] - df['date']).dt.days
    df.drop(columns=['next_shops_closed_date'], inplace=True)
    df['day_after_closing'] = (
        (df['shops_closed'] == 0) & (df['shops_closed'].shift(1) == 1)
    ).astype(int)
    
    df['long_weekend'] = (
        (df['shops_closed'] == 1) & (df['shops_closed'].shift(1) == 1)
    ).astype(int)
    
    df['weekday'] = df['date'].dt.weekday 
    return df
dfs = ['Frankfurt_1', 'Prague_2', 'Brno_1', 'Munich_1', 'Prague_3', 'Prague_1', 'Budapest_1']
processed_dfs = [process_calendar(globals()[df]) for df in dfs]
calendar_extended = pd.concat(processed_dfs).sort_values('date').reset_index(drop=True)

In [6]:
#合併數據集
train_calendar = train.merge(calendar_extended, on=['date', 'warehouse'], how='left')
train_inventory = train_calendar.merge(inventory, on=['unique_id', 'warehouse'], how='left')
train_data = train_inventory.merge(weights, on=['unique_id'], how='left')

test_calendar = test.merge(calendar_extended, on=['date', 'warehouse'], how='left')
test_datas = test_calendar.merge(inventory, on=['unique_id', 'warehouse'], how='left')

train_data = train_data.drop(columns=['availability'])

train_data.dropna(subset=['sales'], inplace=True)

In [7]:
#處理特徵
df=train_data
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['weekday'] = df['date'].dt.weekday
df['dayofweek'] = df['date'].dt.dayofweek
df['weekofyear'] = df['date'].dt.isocalendar().week
df['dayofyear'] = df['date'].dt.dayofyear
df['is_month_start'] = df['date'].dt.is_month_start
df['is_month_end'] = df['date'].dt.is_month_end
df['quarter'] = df['date'].dt.quarter
df["total_dic"]=df['type_0_discount']+df['type_0_discount']+df['type_1_discount']+df['type_2_discount']+df['type_3_discount']+df['type_4_discount']+df['type_5_discount']+df['type_6_discount']
df['total_orders_']=df['total_orders']/df['sell_price_main']
df['total_orders_dic']=df['total_orders_']/df["total_dic"]
df['total_orders_sell_price_main']=df['sell_price_main']/df["total_dic"]
for i in range(7):
    df[f'total_orders{i}']=df[f'type_{i}_discount']/df["total_orders"]
    df[f'total_orders_sell_price_main_{i}']=df[f'type_{i}_discount']/df["total_orders_sell_price_main"]
    df[f'sell_price_main{i}']=df[f'type_{i}_discount']/df["sell_price_main"]
    df[f'sell_price_main_x_{i}']=df[f'type_{i}_discount']/(df["sell_price_main"]*df["total_orders"])
    df[f'total_orders_dic{i}']=df[f'type_{i}_discount']/df["total_orders_dic"]

    df[f'_total_orders{i}']=df[f'type_{i}_discount']*df["total_orders"]
    df[f'_total_orders_sell_price_main_{i}']=df[f'type_{i}_discount']*df["total_orders_sell_price_main"]
    df[f'_sell_price_main{i}']=df[f'type_{i}_discount']*df["sell_price_main"]
    df[f'_total_orders_dic{i}']=df[f'type_{i}_discount']*df["total_orders_dic"]


df.fillna(0, inplace=True)


categorical_columns=['unique_id']+list(df.select_dtypes("object").columns)

for col in categorical_columns:
    df[col] = df[col].astype('category')




df_test=test_datas
df_test['date'] = pd.to_datetime(df_test['date'])

df_test['date'] = pd.to_datetime(df_test['date'])
df_test['year'] = df_test['date'].dt.year
df_test['month'] = df_test['date'].dt.month
df_test['day'] = df_test['date'].dt.day
df_test['weekday'] = df_test['date'].dt.weekday
df_test['dayofweek'] = df_test['date'].dt.dayofweek
df_test['weekofyear'] = df_test['date'].dt.isocalendar().week
df_test['dayofyear'] = df_test['date'].dt.dayofyear
df_test['is_month_start'] = df_test['date'].dt.is_month_start
df_test['is_month_end'] = df_test['date'].dt.is_month_end
df_test['quarter'] = df_test['date'].dt.quarter

df_test["total_dic"]=df_test['type_0_discount']+df_test['type_0_discount']+df_test['type_1_discount']+df_test['type_2_discount']+df_test['type_3_discount']+df_test['type_4_discount']+df_test['type_5_discount']+df_test['type_6_discount']
df_test['total_orders_']=df_test['total_orders']/df_test['sell_price_main']
df_test['total_orders_dic']=df_test['total_orders_']/df_test["total_dic"]
df_test['total_orders_sell_price_main']=df_test['sell_price_main']/df_test["total_dic"]
for i in range(7):
    df_test[f'total_orders{i}']=df_test[f'type_{i}_discount']/df_test["total_orders"]
    df_test[f'total_orders_sell_price_main_{i}']=df_test[f'type_{i}_discount']/df_test["total_orders_sell_price_main"]
    df_test[f'sell_price_main{i}']=df_test[f'type_{i}_discount']/df_test["sell_price_main"]
    df_test[f'sell_price_main_x_{i}']=df_test[f'type_{i}_discount']/(df_test["sell_price_main"]*df_test["total_orders_sell_price_main"])
    df_test[f'total_orders_dic{i}']=df_test[f'type_{i}_discount']/df_test["total_orders_dic"]
    df_test[f'_total_orders{i}']=df_test[f'type_{i}_discount']*df_test["total_orders"]
    df_test[f'_total_orders_sell_price_main_{i}']=df_test[f'type_{i}_discount']*df_test["total_orders_sell_price_main"]
    df_test[f'_sell_price_main{i}']=df_test[f'type_{i}_discount']*df_test["sell_price_main"]
    df_test[f'_total_orders_dic{i}']=df_test[f'type_{i}_discount']*df_test["total_orders_dic"]
df_test.fillna(0, inplace=True)

In [8]:
for col in categorical_columns:
    df_test[col] = df_test[col].astype('category')

In [9]:
# 訓練和測試數據的日期範圍
train_start_date = '2020-08-01'
train_end_date = '2024-03-18'
test_start_date = '2024-03-18'
test_end_date = '2024-06-01'

In [10]:
# 準備訓練和測試數據
X = df.drop(['sales', 'date','weight'], axis=1)
y = df['sales']**(1/8)

In [11]:
train_data = df[(df['date'] < train_end_date)]
test_data = df[(df['date'] >= test_start_date)]

X_train = train_data.drop(['sales', 'date', 'weight'], axis=1)
y_train = train_data['sales']**(1/8)
train_weights = train_data['weight']

X_test = test_data.drop(['sales', 'date', 'weight'], axis=1)
y_test = test_data['sales']**(1/8)

test_weights = test_data['weight']

In [13]:
# 特徵縮放
cols=X.select_dtypes(["int","float"]).columns

In [14]:
from sklearn.preprocessing import RobustScaler
sc = RobustScaler()

for col in cols:
    X_train[col].replace([np.inf, -np.inf], X_train[col].min(), inplace=True)
    X_test[col].replace([np.inf, -np.inf], X_test[col].min(), inplace=True)

    X_train[col].fillna(X_train[col].mean(), inplace=True)
    X_test[col].fillna(X_test[col].mean(), inplace=True)


X_train[col] = sc.fit_transform(X_train[[col]])  
X_test[col] = sc.transform(X_test[[col]])

In [15]:
categorical_feature_indices = [X.columns.get_loc(col) for col in categorical_columns if col in X.columns]

In [16]:
# 訓練LightGBM模型
from lightgbm import log_evaluation, early_stopping
callbacks = [log_evaluation(period=200)]

params = {
    'learning_rate': 0.021796506746095975,  # 學習率
    'num_leaves': 93,  # 樹葉節點數
    'max_depth': 10,  # 樹的最大深度
    'min_child_samples': 25,  # 子葉節點的最小樣本數
    'subsample': 0.7057135664023435,  # 每棵樹的樣本比例
    'colsample_bytree': 0.8528497905459008,  # 每棵樹的特徵比例
    'reg_alpha': 0.036786449788597686,  # L1正則化
    'reg_lambda': 0.3151110021900479,  # L2正則化
    'num_boost_round': 11000,  # 提升迭代次數
    'objective': 'regression',  # 目標函數，這裡是回歸
    'metric': 'mae',  # 評估指標，這裡是平均絕對誤差
    'boosting_type': 'gbdt',  # 提升類型，這裡是梯度提升決策樹
    'verbose': -1  # 設定為-1表示不輸出訓練過程中的信息
}

final_train_dataset = lgb.Dataset(
    X,  # 特徵數據
    label=y,  # 標籤數據
    categorical_feature=categorical_feature_indices,  # 類別特徵的索引
    weight=df['weight']  # 樣本權重
)

final_model = lgb.train(
    params,  # 模型參數
    final_train_dataset,  # 訓練數據集
    num_boost_round=params['num_boost_round'],  # 提升迭代次數
    callbacks=callbacks  # 回調函數
)

final_y_pred = final_model.predict(X_test, num_iteration=final_model.best_iteration)
weighted_mae = np.sum(test_weights * np.abs(y_test**(8) - final_y_pred**(8))) / np.sum(test_weights)

print("\nFinal Model Performance:")
print(f'Weight Mean Absolute Error: {weighted_mae}')

[200]	valid_0's l1: 0.0885557
[400]	valid_0's l1: 0.0820072
[600]	valid_0's l1: 0.0792068
[800]	valid_0's l1: 0.0779112
[1000]	valid_0's l1: 0.0768244
[1200]	valid_0's l1: 0.0756814
[1400]	valid_0's l1: 0.074938
[1600]	valid_0's l1: 0.0744681
[1800]	valid_0's l1: 0.0741401
[2000]	valid_0's l1: 0.0738613
[2200]	valid_0's l1: 0.0735561
[2400]	valid_0's l1: 0.0733461
[2600]	valid_0's l1: 0.0731742
[2800]	valid_0's l1: 0.0729826
[3000]	valid_0's l1: 0.072814
[3200]	valid_0's l1: 0.0727186
[3400]	valid_0's l1: 0.0723859
[3600]	valid_0's l1: 0.0722705
[3800]	valid_0's l1: 0.0721744
[4000]	valid_0's l1: 0.0720761
[4200]	valid_0's l1: 0.071982
[4400]	valid_0's l1: 0.071913
[4600]	valid_0's l1: 0.0718493
[4800]	valid_0's l1: 0.0717889
[5000]	valid_0's l1: 0.0716971
[5200]	valid_0's l1: 0.0716348
[5400]	valid_0's l1: 0.0716016
[5600]	valid_0's l1: 0.0715517
[5800]	valid_0's l1: 0.0715348
[6000]	valid_0's l1: 0.0715413
[6200]	valid_0's l1: 0.0715275
[6400]	valid_0's l1: 0.0715134
[6600]	valid_0's

In [17]:
# 特徵縮放
sc = RobustScaler()

for col in cols:
    X[col].replace([np.inf, -np.inf], X[col].min(), inplace=True)  # 替換無窮大和無窮小值
    df_test[col].replace([np.inf, -np.inf], df_test[col].min(), inplace=True)

    X[col].fillna(X[col].mean(), inplace=True)  # 填充缺失值
    df_test[col].fillna(df_test[col].mean(), inplace=True)

X[cols] = sc.fit_transform(X[cols])  # 對訓練數據進行縮放
df_test[cols] = sc.transform(df_test[cols])  # 對測試數據進行縮放

In [18]:
# 設置LightGBM的回調函數和參數
callbacks = [log_evaluation(period=200)]

params={'learning_rate': 0.021796506746095975,  # 學習率
 'num_leaves': 93,  # 樹葉節點數
 'max_depth': 10,  # 樹的最大深度
 'min_child_samples': 25,  # 子葉節點的最小樣本數
 'subsample': 0.7057135664023435,  # 每棵樹的樣本比例
 'colsample_bytree': 0.8528497905459008,  # 每棵樹的特徵比例
 'reg_alpha': 0.036786449788597686,  # L1正則化
 'reg_lambda': 0.3151110021900479,  # L2正則化
 'num_boost_round': 11000,  # 提升迭代次數
 'objective': 'regression',  # 目標函數，這裡是回歸
 'metric': 'mae',  # 評估指標，這裡是平均絕對誤差
 'boosting_type': 'gbdt',  # 提升類型，這裡是梯度提升決策樹
 'verbose': -1}  # 設定為-1表示不輸出訓練過程中的信息

# 準備LightGBM的訓練數據集
final_train_dataset = lgb.Dataset(X, label=y, 
                                  categorical_feature=categorical_feature_indices,
                                  weight=df['weight'])
# 訓練LightGBM模型
final_model = lgb.train(params, 
                        final_train_dataset, 
                        num_boost_round=params['num_boost_round'],
                       callbacks=callbacks)

In [31]:
# 使用訓練好的模型進行預測
final_y_pred = final_model.predict(df_test.drop(['date'], axis=1), num_iteration=final_model.best_iteration)

In [20]:
# 準備提交結果
sub=df_test.copy()
sub['sales_hat']=final_y_pred**(8)  # 將預測值還原
sub['id']=sub['unique_id'].astype(str) + "_" + sub['date'].astype(str)
sub[['id','sales_hat']].to_csv("submission.csv",index=False)  # 將結果保存為CSV文件

In [24]:
import pandas as pd

# 假設你已經有訓練集和測試集的 DataFrame
train_data = pd.read_csv('data/sales_train.csv')
test_data = pd.read_csv('data/sales_test.csv')

# 計算每個資料集的筆數
train_count = len(train_data)
test_count = len(test_data)
total_count = train_count + test_count

# 建立 DataFrame 來顯示結果
summary_df = pd.DataFrame({
    '資料集': ['訓練資料', '測試資料', '總和'],
    '筆數': [train_count, test_count, total_count]
})

# 顯示結果
print(summary_df)

    資料集       筆數
0  訓練資料  4007419
1  測試資料    47021
2    總和  4054440


In [26]:
import pandas as pd
import lightgbm as lgb
import time
import os

# 讀取訓練資料和測試資料
train_data = pd.read_csv('data/sales_train.csv')
test_data = pd.read_csv('data/sales_test.csv')

# 將日期轉換為數值型特徵
train_data['date'] = pd.to_datetime(train_data['date'])
train_data['year'] = train_data['date'].dt.year
train_data['month'] = train_data['date'].dt.month
train_data['day'] = train_data['date'].dt.day
train_data['day_of_week'] = train_data['date'].dt.dayofweek

# 將類別型特徵轉換為數值型特徵
train_data['warehouse'] = train_data['warehouse'].astype('category').cat.codes

# 訓練數據和標籤
X_train = train_data.drop(columns=['sales', 'date'])
y_train = train_data['sales']

# 設置LightGBM的回調函數和參數
callbacks = [lgb.log_evaluation(period=200)]
params = {
    'learning_rate': 0.021796506746095975,
    'num_leaves': 93,
    'max_depth': 10,
    'min_child_samples': 25,
    'subsample': 0.7057135664023435,
    'colsample_bytree': 0.8528497905459008,
    'reg_alpha': 0.036786449788597686,
    'reg_lambda': 0.3151110021900479,
    'num_boost_round': 11000,
    'objective': 'regression',
    'metric': 'mae',
    'boosting_type': 'gbdt',
    'verbose': -1
}

# 準備LightGBM的訓練數據集
train_dataset = lgb.Dataset(X_train, label=y_train)

# 訓練LightGBM模型並計算訓練時間
start_time = time.time()
model = lgb.train(params, train_dataset, callbacks=callbacks)
end_time = time.time()
training_time = end_time - start_time

# 保存模型到文件並計算模型大小
model_file = 'lightgbm_model.txt'
model.save_model(model_file)
model_size = os.path.getsize(model_file) / 1024  # 以KB為單位

# 模型出處
model_source = 'LightGBM'

# 建立 DataFrame 來顯示結果
summary_df = pd.DataFrame({
    '項目': ['模型出處', '模型大小 (KB)', '訓練時間 (秒)'],
    '數值': [model_source, model_size, training_time]
})

# 顯示結果
print(summary_df)

          項目            數值
0       模型出處      LightGBM
1  模型大小 (KB)  87755.418945
2   訓練時間 (秒)    341.673715
